In [31]:
import numpy as np
import scipy as sp
import scipy.stats
import matplotlib.pyplot as plt
import pandas as pd

AttributeError: module 'matplotlib.cbook' has no attribute '_define_aliases'

In [2]:
S0 = 10000
M = 256
r = 0.05
sigma = 0.40
T = 1

In [3]:
def stockprice_simulation(S0, T, sigma, r, M, seed=None):
    if isinstance(sigma, np.ndarray):
        if len(sigma) >= M:
            sigma_vec = sigma[:M]
        else:
            raise ValueError("dimension error!")
    else:
        sigma_vec = sigma * np.ones(M + 1)
    if seed is not None:
        np.random.seed(seed)
    dt = T / M
    T_vec = np.linspace(T, 0, M + 1)
    S_vec = np.zeros(M + 1)
    S_vec[0] = S0
    for t in range(1, M + 1):
        S_vec[t] = S_vec[t - 1] * np.exp((r - 0.5 * sigma_vec[t - 1] ** 2) * dt 
                 + sigma_vec[t - 1] * np.sqrt(dt) * np.random.standard_normal())
    S_vec = np.around(S_vec)
    return S_vec.T, T_vec, sigma_vec

In [4]:
S_vec, T_vec, sigma_vec = stockprice_simulation(S0, T, sigma, r, M, seed=3)

In [5]:
def call_value(S, T, sigma, r, K):
    if not hasattr(T, "__len__") and T == 0:
        return np.maximum(S - K, 0)
    d1 = ((np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T)))
    d2 = ((np.log(S / K) + (r - 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T)))
    value = (S * sp.stats.norm.cdf(d1, 0, 1) - K * np.exp(-r * T) * sp.stats.norm.cdf(d2, 0, 1))
    return value

In [6]:
def call_delta(S, T, sigma, r, K):
    if not hasattr(T, "__len__") and T == 0:
        return int(S > K)
    d1 = ((np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T)))
    delta = sp.stats.norm.cdf(d1, 0, 1)
    return delta

In [7]:
K = 10000
NomialAmount = 10

In [13]:
V_vec = NomialAmount * call_value(S_vec, T_vec, sigma_vec, r, K)
D_vec = NomialAmount * call_delta(S_vec, T_vec, sigma_vec, r, K)

C:\Users\MinjungChoi\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in true_divide
  after removing the cwd from sys.path.
C:\Users\MinjungChoi\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in true_divide
  """


In [20]:
plt.subplot(311)
plt.plot(S_vec, 'r-')
plt.subplot(312)
plt.plot(V_vec, 'g-')
plt.subplot(313)
plt.plot(D_vec, 'b-')
plt.show()

AttributeError: module 'matplotlib' has no attribute 'subplot'

In [29]:
maturities = np.linspace(0, 1, 13)
prices = np.linspace(8000, 12000, 101)
_T, S = np.meshgrid(maturities, prices)
V = np.zeros_like(S)
for i, s in enumerate(prices):
    for j, t in enumerate(maturities):
        V[i, j] = NomialAmount * call_value(s, t, sigma, r, K)

plt.plot(prices, V, alpha=0.5)
plt.xlabel('prices')
plt.ylabel('European call value')
plt.legend(["%d month" % i for i in range(13)], loc=2)
plt.hold(True)
plt.scatter(S_vec, V_vec, c=T_vec, cmap=mpl.cm.jet_r)
plt.plot(S_vec, V_vec, c="k", alpha=0.5)
plt.show()

AttributeError: module 'matplotlib' has no attribute 'pyplot'